In [3]:
import pickle

import pandas as pd
from pygov_br.camara_deputados import cd

In [4]:
years = range(2000, 2019)

In [55]:
df_data = []

for year in years:
    print ('Processing year %s' % year)
    speeches = cd.sessions.speeches('01/01/%d' % year, '20/12/%d' % year, parliamentary_name='Jair Bolsonaro')
    print ('Found %d speeches' % len(speeches))
    
    for speach in speeches:
        codigo = speach['codigo']
        data = speach['data'].strftime("%d-%m-%Y")
        tipo = speach['tipo']

        tipo_sessao = speach['fasesSessao']['faseSessao']['descricao']

        discursos = speach['fasesSessao']['faseSessao']['discursos']['discurso']
        
        if isinstance(discursos, dict):
            discursos = [discursos]

        for discurso in discursos:
        
            numero_orador = discurso['orador']['numero']

            partido_orador = discurso['orador']['partido']

            numero_quarto = discurso['numeroQuarto']
            numero_insercao = discurso['numeroInsercao']

            sumario = discurso['sumario']
            tags = discurso['txtIndexacao']

            df_data.append(
                [codigo, data, tipo, tipo_sessao, numero_orador, partido_orador, numero_quarto, numero_insercao, sumario, tags]
            )

Processing year 2000
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/ListarDiscursosPlenario?dataIni=01/01/2000&dataFim=20/12/2000&codigoSessao=&parteNomeParlamentar=Jair Bolsonaro&siglaPartido=&siglaUF= {}
Found 8 speeches
Processing year 2001
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/ListarDiscursosPlenario?dataIni=01/01/2001&dataFim=20/12/2001&codigoSessao=&parteNomeParlamentar=Jair Bolsonaro&siglaPartido=&siglaUF= {}
Found 51 speeches
Processing year 2002
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/ListarDiscursosPlenario?dataIni=01/01/2002&dataFim=20/12/2002&codigoSessao=&parteNomeParlamentar=Jair Bolsonaro&siglaPartido=&siglaUF= {}
Found 27 speeches
Processing year 2003
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/ListarDiscursosPlenario?dataIni=01/01/2003&dataFim=20/12/2003&codigoSessao=&parteNomeParlamentar=Jair Bolsonaro&siglaPartido=&siglaUF= {}
Found 55 speeches
Processing year 2004
http://www.camara.leg.br/SitCamaraWS/Ses

In [70]:
speeches_df = pd.DataFrame(df_data, columns=['codigo', 'data', 'tipo', 'tipo_sessao', 'numero_orador',
                           'partido_orador', 'numero_quarto', 'numero_insercao', 'sumario', 'tags'])
speeches_df.data = pd.to_datetime(speeches_df.data)
speeches_df.head()

,codigo,data,tipo,tipo_sessao,numero_orador,partido_orador,numero_quarto,numero_insercao,sumario,tags
0,234.2.51.O,13-12-2000,Ordinária - CD,Pequeno Expediente,2,PPB,7,0,Defesa de rejeição da medida provisória conces...,"MEDIDA PROVISÓRIA, REAJUSTE, SALÁRIO, MILITAR,..."
1,227.2.51.O,07-12-2000,Extraordinária - CD,Breves Comunicações,1,PPB,5,0,Aplausos ao Deputado Paulo Paim pela atuação e...,"ELOGIO, PRONUNCIAMENTO, PAULO PAIM, DEPUTADO F..."
2,220.2.51.O,30-11-2000,Extraordinária - CD,Breves Comunicações,2,PPB,18,0,"Certeza da punição, pela Marinha do Brasil, de...","IMPROPRIEDADE, DENÚNCIA, VANESSA GRAZZIOTIN, D..."
3,219.2.51.O,29-11-2000,Extraordinária - CD,Breves Comunicações,4,PPB,35,0,Contrariedade dos servidores públicos militare...,"CRÍTICA, GOVERNO FEDERAL, FERNANDO HENRIQUE CA..."
4,211.2.51.O,22-11-2000,Ordinária - CD,Pequeno Expediente,3,PPB,21,0,Repúdio à nova proposta apresentada à Lei de R...,"VOTO CONTRÁRIO, CRIAÇÃO, MINISTÉRIO DA DEFESA...."


In [73]:
speeches_df.to_excel('list-discursos.xlsx')

In [81]:
tags_data = []
for idx, row in speeches_df.iterrows():
    tags = [tag.strip().replace('.', '') for tag in row['tags'].split(',')]
    for tag in tags:
        tags_data.append([idx, tag])

In [84]:
tags_df = pd.DataFrame(tags_data, columns=['speach_id', 'tag'])
tags_df.head()

,speach_id,tag
0,0,MEDIDA PROVISÓRIA
1,0,REAJUSTE
2,0,SALÁRIO
3,0,MILITAR
4,0,PREJUÍZO


In [86]:
tags_df.to_excel('tags-discursos.xlsx', index=False)

In [98]:
topic_data = []
for idx, row in speeches_df.iterrows():
    discurso = cd.sessions.full_speech(row.codigo, row.numero_orador, row.numero_quarto, row.numero_insercao)['discurso']
    topic_data.append([idx, discurso])

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=234.2.51.O&numOrador=2&numQuarto=7&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=227.2.51.O&numOrador=1&numQuarto=5&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=220.2.51.O&numOrador=2&numQuarto=18&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=219.2.51.O&numOrador=4&numQuarto=35&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=211.2.51.O&numOrador=3&numQuarto=21&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=201.2.51.O&numOrador=3&numQuarto=81&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTe

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=034.3.51.O&numOrador=1&numQuarto=5&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=032.3.51.O&numOrador=2&numQuarto=4&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=026.3.51.O&numOrador=3&numQuarto=11&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=019.3.51.O&numOrador=2&numQuarto=76&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=017.3.51.O&numOrador=4&numQuarto=10&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=006.3.51.O&numOrador=2&numQuarto=2&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeo

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=238.1.52.O&numOrador=2&numQuarto=64&numInsercao=14 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=238.1.52.O&numOrador=2&numQuarto=64&numInsercao=61 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=227.1.52.O&numOrador=1&numQuarto=14&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=210.1.52.O&numOrador=2&numQuarto=6&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=191.1.52.O&numOrador=2&numQuarto=8&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=189.1.52.O&numOrador=1&numQuarto=46&numInsercao=102 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterIntei

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=228.2.52.O&numOrador=1&numQuarto=144&numInsercao=25 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=222.2.52.O&numOrador=1&numQuarto=18&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=221.2.52.O&numOrador=1&numQuarto=20&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=221.2.52.O&numOrador=2&numQuarto=33&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=009.2.52.N&numOrador=0&numQuarto=0&numInsercao=17 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=187.2.52.O&numOrador=2&numQuarto=114&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInte

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=170.3.52.O&numOrador=2&numQuarto=23&numInsercao=159 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=169.3.52.O&numOrador=2&numQuarto=10&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=169.3.52.O&numOrador=3&numQuarto=87&numInsercao=44 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=165.3.52.O&numOrador=1&numQuarto=96&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=148.3.52.O&numOrador=1&numQuarto=24&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=148.3.52.O&numOrador=2&numQuarto=32&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInte

...

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=165.3.54.O&numOrador=2&numQuarto=38&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=155.3.54.O&numOrador=2&numQuarto=3&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=155.3.54.O&numOrador=1&numQuarto=10&numInsercao=7 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=155.3.54.O&numOrador=1&numQuarto=10&numInsercao=13 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=154.3.54.O&numOrador=1&numQuarto=1&numInsercao=5 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=154.3.54.O&numOrador=1&numQuarto=1&numInsercao=6 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTe

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=181.4.54.O&numOrador=2&numQuarto=51&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=171.4.54.O&numOrador=2&numQuarto=94&numInsercao=69 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=158.4.54.O&numOrador=2&numQuarto=9&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=158.4.54.O&numOrador=1&numQuarto=27&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=155.4.54.O&numOrador=1&numQuarto=1&numInsercao=18 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=148.4.54.O&numOrador=2&numQuarto=56&numInsercao=15 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteir

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=274.1.55.O&numOrador=3&numQuarto=18&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=274.1.55.O&numOrador=2&numQuarto=64&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=267.1.55.O&numOrador=1&numQuarto=9&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=256.1.55.O&numOrador=1&numQuarto=18&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=255.1.55.O&numOrador=1&numQuarto=1&numInsercao=1 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=255.1.55.O&numOrador=1&numQuarto=1&numInsercao=2 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeo

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=008.1.55.O&numOrador=1&numQuarto=75&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=340.2.55.O&numOrador=1&numQuarto=50&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=322.2.55.O&numOrador=2&numQuarto=1&numInsercao=43 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=322.2.55.O&numOrador=2&numQuarto=1&numInsercao=60 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=321.2.55.O&numOrador=2&numQuarto=11&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=321.2.55.O&numOrador=3&numQuarto=49&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiro

http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=293.3.55.O&numOrador=2&numQuarto=15&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=288.3.55.O&numOrador=2&numQuarto=33&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=288.3.55.O&numOrador=2&numQuarto=73&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=280.3.55.O&numOrador=1&numQuarto=18&numInsercao=43 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=199.3.55.O&numOrador=2&numQuarto=22&numInsercao=291 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInteiroTeorDiscursosPlenario?codSessao=156.3.55.O&numOrador=1&numQuarto=70&numInsercao=0 {}
http://www.camara.leg.br/SitCamaraWS/SessoesReunioes.asmx/obterInte

In [99]:
discursos_df = pd.DataFrame(topic_data, columns=['speach_id', 'discurso'])
discursos_df.head()

,speach_id,discurso
0,0,O SR. JAIR BOLSONARO (PPB-RJ. Sem revisão do o...
1,1,O SR. JAIR BOLSONARO (PPB-RJ. Sem revisão do o...
2,2,O SR. JAIR BOLSONARO (PPB-RJ. Sem revisão do o...
3,3,O SR. JAIR BOLSONARO (PPB-RJ. Sem revisão do o...
4,4,O SR. JAIR BOLSONARO (PPB-RJ. Sem revisão do o...


In [100]:
discursos_df.to_excel('discursos.xlsx', index=False)

In [179]:
docs = discursos_df.discurso.tolist()

In [180]:
with open('docs.pkl', 'wb') as _f:
    pickle.dump(docs, _f)

### Topic model

In [29]:
with open('docs.pkl', 'rb') as _f:
    docs = pickle.load(_f)

In [106]:
import codecs

stop_words = codecs.open('stopwords-pt.txt', encoding='utf-8').read().split()

In [107]:
import sklearn
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS

for _ in ['tinha', 'estamos', 'tínhamos', 'v.exa', 's.exa', 'lhe', 'vejam'] + stop_words:
    STOP_WORDS.add(_)

nlp = spacy.load('pt_core_news_sm')

In [33]:
len(docs)

931

In [35]:
docs[900]

'O SR. JAIR BOLSONARO (Bloco/PSC-RJ. Sem revisão do orador.) - Sr. Presidente, quero cumprimentar aqui os meus colegas da Polícia Militar do Distrito Federal e os integrantes do Corpo de Bombeiros Militar, contrários ao Projeto nº 3.123, que não tem nada a ver com o teto de vocês ao não transformar a licença em pecúnia. É um crime retirar qualquer direito de policiais e bombeiros militares. (Palmas nas galerias.)\nQuero dizer mais, Sr. Presidente, desde agora - eu não sou o Coronel Alberto Fraga, que é muito mais chegado a vocês do que eu -, que, no dia da votação do impeachment aqui, no final de abril, vocês todos têm que sitiar o Congresso Nacional e botar lá fora 50 mil pessoas ou mais!\nAquele Parlamentar que, porventura, não votar pela saída dessa impostora e corrupta, Dilma Rousseff, vai ter problemas para sair do Congresso. Então, estejam todos aqui no final de abril para votar o impeachment de Dilma Rousseff!\nParabéns a vocês, que estão aqui no Congresso defendendo os seus dir

Todo texto começa com a mesma sentença, vamos tirar tudo até "Sr. Presidente" e tirar o restante de "Sr. Presidente"

In [36]:
docs = [doc[78:].replace("Sr. Presidente", "") for doc in docs]

#### Parse

In [37]:
%%time

docs = [nlp(doc) for doc in docs]

Wall time: 5min 21s


#### Entities

In [50]:
def ent_is_noise(ent):
    if len(ent) < 3 or ent.lower() in STOP_WORDS:
        return True
    return False

In [54]:
ent_labels = {
    'LOC': 'Local',
    'ORG': 'Organização',
    'PER': 'Pessoa',
    'MISC': 'Outros'
}

In [57]:
ents_data = []
for speach_id, doc in enumerate(docs):
    for ent in doc.ents:
        if ent_is_noise(ent.text):
            continue
        ents_data.append((speach_id, ent.text, ent_labels[ent.label_]))

In [58]:
ents_df = pd.DataFrame(ents_data, columns=['speach_id', 'entidade', 'tipo_entidade'])
ents_df.head()

,speach_id,entidade,tipo_entidade
0,0,Nioaque,Local
1,0,Mato Grosso do Sul,Local
2,0,Exército brasileiro,Organização
3,0,Tenente,Pessoa
4,0,Cumpri,Pessoa


In [60]:
ents_df.to_excel('entidades.xlsx', index=False)

#### Topic model

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF     

In [108]:
def filter_doc(doc):
    tokens = []
    for token in doc:
        if token.is_stop or token.text.lower() in STOP_WORDS or len(token.text) < 3 or token.is_digit:
            continue
        tokens.append(token.text)
    return tokens

In [133]:
tfidf_vectorizer = TfidfVectorizer(
    #max_df=1, min_df=1,
    max_features=10000, norm='l2',
    tokenizer=filter_doc, lowercase=False
)
tfidf = tfidf_vectorizer.fit_transform(docs)
terms = tfidf_vectorizer.get_feature_names()

In [134]:
import operator

weights = {}
for col, term in enumerate(terms):
    weights[term] = tfidf.sum(axis=0)[0, col]

ranking = sorted(weights.items(), key=operator.itemgetter(1), reverse=True)

for i, pair in enumerate(ranking[0:20]):
    print( "%02d. %s (%.2f)" % ( i+1, pair[0], pair[1] ))

01. Deputado (31.28)
02. militares (31.08)
03. Forças (25.66)
04. Armadas (24.80)
05. Governo (24.50)
06. Ministro (22.08)
07. Casa (21.59)
08. Presidente (21.07)
09. Brasil (20.60)
10. projeto (20.52)
11. Exército (19.35)
12. militar (19.24)
13. Comissão (18.29)
14. reais (17.86)
15. Dilma (16.70)
16. País (16.57)
17. Defesa (15.70)
18. Rio (14.86)
19. voto (14.49)
20. Federal (14.36)


In [135]:
tfidf.shape

(931, 10000)

In [143]:
no_topics = 50

# Run NMF
model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
W = model.fit_transform(tfidf)
H = model.components_

In [144]:
for topic_index in range(no_topics):
    
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    
    _terms = [terms[term_index] for term_index in top_indices[0:10]]    
    str_descriptor = ", ".join(["%s" % t for t in _terms])
    
    print("Topic %02d: %s" % (topic_index, str_descriptor))

Topic 00: Forças, Armadas, militares, militar, Deputado, Casa, Exército, Governo, medida, provisória
Topic 01: útil, cobrar, cofres, cofre, coerência, coerente, cocaína, cobrou, cobre, cobrados
Topic 02: sessão, Sr., palavra, convidou, concedeu, Srs, homenagens, Mesa, Presidência, Sessão
Topic 03: Dilma, Rousseff, marido, VPR, Carlos, Baixada, Cuba, Araújo, Palmares, Sra.
Topic 04: gay, kit, LGBT, escolas, Haddad, homossexual, Humanos, livros, Direitos, sexual
Topic 05: Quintão, Geraldo, Dr., Defesa, Ministro, Cardoso, Henrique, Fernando, Ministério, generais
Topic 06: projeto, taifeiros, urgência, lei, Aeronáutica, Senado, aprovado, aprovação, Líderes, Maia
Topic 07: voto, impresso, urna, eletrônica, eleições, fraude, TSE, votos, partido, suspeição
Topic 08: Ministro, José, Alencar, Viegas, Defesa, reajuste, Lula, militares, Ministério, Presidente
Topic 09: habitantes, milhões, planejamento, familiar, laqueadura, vasectomia, Brasil, natalidade, paternidade, população
Topic 10: Bolsa, 

In [145]:
topic_data = []

for topic_index in range(no_topics):
    
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    
    _terms = [terms[term_index] for term_index in top_indices[0:10]]
    _weights = [H[topic_index,term_index] for term_index in top_indices[0:10]]
    
    for t, w in zip(_terms, _weights):
        topic_data.append([topic_index, t, w])

In [146]:
topic_df = pd.DataFrame(topic_data, columns=['topic_id', 'palavra', 'peso'])
topic_df.head()

,topic_id,palavra,peso
0,0,Forças,0.602176
1,0,Armadas,0.582740
2,0,militares,0.546655
3,0,militar,0.348582
4,0,Deputado,0.338095


In [147]:
topic_df.to_excel('topics.xlsx', index=False)

In [149]:
topic_doc_data = []
for topic_index in range(no_topics):
    
    top_indices = np.argsort(W[:,topic_index])[::-1]    
    
    _docs = [doc_index for doc_index in top_indices[0:10]]
    _weights = [W[doc_index, topic_index] for doc_index in top_indices[0:10]]
    
    for d, w in zip(_docs, _weights):
        topic_doc_data.append([topic_index, d, w])

In [150]:
topic_df = pd.DataFrame(topic_doc_data, columns=['topic_id', 'speach_id', 'peso'])
topic_df.head()

,topic_id,speach_id,peso
0,0,466,0.222010
1,0,318,0.209759
2,0,132,0.205128
3,0,173,0.204509
4,0,877,0.198607


In [151]:
topic_df.to_excel('topics-docs.xlsx', index=False)